In [1]:
import pandas as pd
import numpy as np
import math
import os
from collections import OrderedDict
from IPython.display import display, clear_output, Markdown, HTML

from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.io as pio

from simulator import *
from simulator_plotting import *

init_notebook_mode(connected=True)

In [2]:
def plot_simulation2(results, genotypes_to_plot=[], carrying_capacity=9):
    trace = results['trace']
    if not genotypes_to_plot:
        genotypes_to_plot = list(
            set(results['greedy_path']) | set(results['actual_path'])
        )
        genotypes_to_plot.sort(key=lambda g: np.argmax(trace[g] > 0))
    y_max = carrying_capacity + 1
    timesteps = len(next(iter(trace.values())))
    #           length of an arbitrary entry from dictionary
    index = np.array(range(0, timesteps), np.int64)
    
    # data
    data = []
    for g in genotypes_to_plot:
        data.append(go.Scatter(
            x=index,
            y=trace[g],
            name=g,
            line=dict(width=1)
        ))

    # vertical lines
    for crit in ['T_1', 'T_d', 'T_f']:
        if results[crit] != -1:
            data.append(go.Scatter(
                x=[results[crit], results[crit]],
                y=[0, 10**y_max],
                line=dict(color='black', width=1),
                name = crit,
                mode = 'lines',
                showlegend = False
            ))

    # vertical line annotations
    vlines = []
    for crit, text in zip(['T_1', 'T_d', 'T_f'],
                          ['<i>T</i><sub>1</sub>',
                           '<i>T</i><sub>d</sub>',
                           '<i>T</i><sub>f</sub>']):
        if results[crit] != -1:
            vlines.append(dict(
                x=results[crit],
                y=y_max*.975,
                xref='x',
                yref='y',
                text=text,
                showarrow=False,
                xanchor='left'
            ))
        
    # layout
    layout = go.Layout(
        xaxis=dict(
            title='Timestep'
        ),
        yaxis=dict(
            type='log',
            range=[0,y_max],
            exponentformat='power',
            title='Abundance of each genotype',
            tickfont=dict(size=10)
        ),
        width=600, #600
        height=400, #400
        margin=dict(t=30, b=40, l=50, r=0),
        annotations=vlines
    )
        
    fig = go.Figure(data=data, layout=layout)
    return fig

In [3]:
pairs = [('CTX', 'SAM'), ('AM', 'AMC'), ('ZOX', 'CXM')]
pairs = [[dataset2.loc[item].tolist() for item in pair] for pair in pairs]

In [102]:
# CTX and SAM switching
plot_simulation(simulate(pairs[0], frequency=100))

In [131]:
# CTX alone
plot_simulation(simulate(dataset2.loc['CTX'].tolist(), timesteps=10))

In [6]:
# CTX and SAM one switch only
results = simulate(pairs[0], durations=[75, 1125])
plot = plot_simulation2(results)
plot['layout'].update(title='CTX + SAM with a single switch')
iplot(plot, show_link=False)
pio.write_image(plot, 'report/figx4/CTX+SAM.png', scale=3)
results['T_f']

124

In [78]:
# CTX and SAM switching
plot_simulation(simulate(pairs[2], frequency=200))

In [4]:
# ZOX and CXM
results = simulate(pairs[2], durations=[50, 1150])
plot = plot_simulation2(results)
plot['layout'].update(title='ZOX + CXM with a single switch')
iplot(plot, show_link=False)
pio.write_image(plot, 'report/figx4/ZOX+CXM.png', scale=3)
results['T_f']

76

In [23]:
# AM and AMC
sim = simulate(pairs[1], frequency=100)
plot_simulation(sim, ['0000', '0010', '1010', '0011', '1011', '1111', '1101'])

In [68]:
# AM alone
sim = simulate(dataset2.loc['AM'].tolist())
plot_simulation(sim)

In [69]:
# AMC alone
sim = simulate(dataset2.loc['AMC'].tolist())
plot_simulation(sim)

In [5]:
# 3 switches
AM = dataset2.loc['AM'].tolist()
AMC = dataset2.loc['AMC'].tolist()

results = simulate([AM, AMC, AM, AMC, AM, AMC], durations=[75, 75, 75, 75, 75, 1000])
plot = plot_simulation2(results)
plot['layout'].update(title='AM + AMC with 5 switches')
iplot(plot, show_link=False)
pio.write_image(plot, 'report/figx4/AM+AMC.png', scale=3)
results['T_f']

439

In [5]:
plot_simulation(simulate([AM, AMC], frequency=10, carrying_cap=10**4)) # the 4th switch is unnecessary and actually slows T_f down

In [101]:
dataset2.loc['AMC']

Genotype
0000    1.435
0001    1.573
0010    1.061
0011    1.457
0100    1.672
0101    1.625
0110    0.073
0111    0.068
1000    1.417
1001    1.351
1010    1.538
1011    1.590
1100    1.377
1101    1.914
1110    1.307
1111    1.728
Name: AMC, dtype: float64